# ETF相关知识

## 国内ETF一共有多少只？其中已经上市的ETF数量是多少？
* 上交所
    * ETF列表：http://www.sse.com.cn/assortment/fund/etf/list/
    * 指数对应的成分股：http://www.sse.com.cn/assortment/fund/list/etfinfo/redemptionlist/index.shtml?FUNDID=510630
* 深交所
    * ETF列表：http://www.szse.cn/market/fund/list/etfList/index.html
    * 指数与样本股：http://www.szse.cn/market/exponent/sample/index.html

# 获取ETF列表
本来想着是直接爬取，结果受挫。。。
* 上交所的爬取异常，直接拷贝的
* 深交所的直接提供下载

## 上交所

In [8]:
# import requests
# from pyquery import PyQuery as pq
# url = 'http://www.sse.com.cn/assortment/fund/etf/list/'
# headers = {
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36 Maxthon/5.2.1.6000',
# }
# url = 'http://query.sse.com.cn/commonQuery.do?jsonCallBack=jsonpCallback12716&isPagination=true&pageHelp.pageSize=25&pageHelp.pageNo=1&pageHelp.beginPage=1&pageHelp.cacheSize=1&pageHelp.endPage=11&sqlId=COMMON_SSE_ZQPZ_ETFLB_L_NEW&type=1'
# r = requests.get(url, headers=headers)
# r.encoding='utf8'
# r.text

In [37]:
import pandas as pd

In [38]:
with open('etflist_shanghai.txt') as f:
    ll = f.readlines()

s = pd.Series(ll)

df = pd.DataFrame(s[s.index%7==0])

df.reset_index(drop=True, inplace=True)

df.columns = ['code']

df.loc[:, 'code'] = df.code.str.strip()

df.loc[:, 'name'] = s[s.index%7==2].tolist()

df.loc[:, 'name'] = df.name.str.strip()

In [39]:
df.loc[:, 'company'] = s[s.index%7==4].tolist()

df.loc[:, 'company'] = df.company.str.strip()

In [40]:
df['source']='sh'

In [41]:
df.head(2)

,code,name,company,source
0,510010,治理ETF,交银施罗德基金管理有限公司,sh
1,510020,超大ETF,博时基金管理有限公司,sh


## 深交所

In [42]:
df_sz = pd.read_excel('ETFlist_sz.xlsx')

In [43]:
df_sz.head()

,证券代码,证券简称,拟合指数,当前规模(份),基金管理人
0,159979,湾创100,980001,"244,928,947",广发基金管理有限公司
1,159995,芯片ETF,980017,"4,893,463,006",华夏基金管理有限公司
2,159993,龙头券商,,"1,053,697,982",鹏华基金管理有限公司
3,159986,消费100,399364 消费100,"211,152,218",弘毅远方基金管理有限公司
4,159990,小盘价值,399377 小盘价值,"433,419,629",银华基金管理股份有限公司


In [44]:
df_sz['source']='sz'

In [45]:
df_sz = df_sz[['证券代码', '证券简称','基金管理人','source']]

In [46]:
df_sz.columns = df.columns.tolist()

In [47]:
df_sz.head(2)

,code,name,company,source
0,159979,湾创100,广发基金管理有限公司,sz
1,159995,芯片ETF,华夏基金管理有限公司,sz


In [48]:
dfc = pd.concat([df, df_sz], ignore_index=True)
dfc.head(2)

,code,name,company,source
0,510010,治理ETF,交银施罗德基金管理有限公司,sh
1,510020,超大ETF,博时基金管理有限公司,sh


In [49]:
dfc.shape

(259, 4)

In [50]:
dfc.loc[:, 'code'] = dfc.source+dfc.code.map(str)

In [51]:
dfc.to_pickle('etflist_sh_sz.pkl')

In [52]:
dfc.head(2)

,code,name,company,source
0,sh510010,治理ETF,交银施罗德基金管理有限公司,sh
1,sh510020,超大ETF,博时基金管理有限公司,sh


# 获取行情

In [53]:
import pandas as pd

In [54]:
dfc=pd.read_pickle('etflist_sh_sz.pkl')

In [55]:
from regular_investing_utils import get_hist_k_data

In [56]:
def ft(s):
    return (s[-1]-s[0])/s[0]

In [65]:
dfc.loc[dfc.code=='sh513100']

,code,name,company,source,5_pct,20_pct,60_pct,120_pct,250_pct,close,volume,amount
158,sh513100,纳指ETF,国泰基金管理有限公司,sh,-0.09483,0.114076,0.248029,0.35292,0.557606,3.799,65103993.0,2.473301e+08


In [73]:
dfc.iloc[232:]

,code,name,company,source,5_pct,20_pct,60_pct,120_pct,250_pct,close,volume,amount
232,sz159932,500深ETF,大成基金管理有限公司,sz,0.048493,0.041667,0.161946,0.206637,0.346801,1.6,125200.0,200320.0
233,sz159933,金地ETF,国投瑞银基金管理有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,sz159928,消费ETF,汇添富基金管理股份有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,sz159929,医药ETF,汇添富基金管理股份有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,sz159930,能源ETF,汇添富基金管理股份有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,sz159931,金融ETF,汇添富基金管理股份有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,sz159926,国债ETF,嘉实基金管理有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,sz159925,南方300,南方基金管理股份有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,sz159922,500ETF,嘉实基金管理有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,sz159923,100ETF,大成基金管理有限公司,sz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
dfc.code.tolist()[81:]

['sh512560',
 'sh512570',
 'sh512580',
 'sh512590',
 'sh512600',
 'sh512610',
 'sh512640',
 'sh512650',
 'sh512660',
 'sh512670',
 'sh512680',
 'sh512690',
 'sh512700',
 'sh512710',
 'sh512720',
 'sh512730',
 'sh512750',
 'sh512760',
 'sh512770',
 'sh512780',
 'sh512790',
 'sh512800',
 'sh512810',
 'sh512820',
 'sh512850',
 'sh512860',
 'sh512870',
 'sh512880',
 'sh512890',
 'sh512900',
 'sh512910',
 'sh512920',
 'sh512930',
 'sh512950',
 'sh512960',
 'sh512970',
 'sh512980',
 'sh512990',
 'sh515000',
 'sh515010',
 'sh515020',
 'sh515050',
 'sh515060',
 'sh515070',
 'sh515080',
 'sh515110',
 'sh515150',
 'sh515180',
 'sh515200',
 'sh515300',
 'sh515310',
 'sh515330',
 'sh515350',
 'sh515360',
 'sh515390',
 'sh515520',
 'sh515550',
 'sh515580',
 'sh515590',
 'sh515600',
 'sh515650',
 'sh515660',
 'sh515680',
 'sh515700',
 'sh515750',
 'sh515800',
 'sh515810',
 'sh515860',
 'sh515870',
 'sh515880',
 'sh515900',
 'sh515980',
 'sh515990',
 'sh510900',
 'sh513000',
 'sh513030',
 'sh513050',

In [75]:
for code in dfc.code.tolist()[233:]:
    print(code)
    df = get_hist_k_data(code, 1500)
    for time_range in [5, 20, 60, 120, 250]:
        dfc.loc[dfc.code==code, f'{time_range}_pct']=df.close.rolling(time_range).agg(ft).iloc[-1]
    dfc.loc[dfc.code==code, f'close']=df.close.iloc[-1]
    dfc.loc[dfc.code==code, f'volume']=int(df.volume.iloc[-1])
    dfc.loc[dfc.code==code, f'amount']=dfc.loc[dfc.code==code, f'close']*dfc.loc[dfc.code==code, f'volume']
    
    df.to_pickle(f'fund_daily_data/{code}_190219_1500.pkl')

sz159933
sz159928
sz159929
sz159930
sz159931
sz159926
sz159925
sz159922
sz159923
sz159920
sz159919
sz159918
sz159915
sz159912
sz159913
sz159916
sz159911
sz159910
sz159909
sz159907
sz159908
sz159906
sz159905
sz159903
sz159902
sz159901


In [77]:
dfc.to_pickle('etflist_sh_sz_190219.pkl')

In [76]:
dfc.tail()

,code,name,company,source,5_pct,20_pct,60_pct,120_pct,250_pct,close,volume,amount
254,sz159906,深成长,大成基金管理有限公司,sz,0.038563,0.104380,0.286645,0.384346,0.750369,1.185,8246700.0,9.772340e+06
255,sz159905,深红利,工银瑞信基金管理有限公司,sz,0.025349,-0.057985,0.007626,0.066738,0.332885,1.982,32022600.0,6.346879e+07
256,sz159903,深成ETF,南方基金管理股份有限公司,sz,0.036975,0.019835,0.135235,0.193424,0.429896,1.234,1169284.0,1.442896e+06
257,sz159902,中 小 板,华夏基金管理有限公司,sz,0.030743,0.039323,0.160949,0.255548,0.431791,3.621,14791595.0,5.356037e+07
258,sz159901,深100ETF,易方达基金管理有限公司,sz,0.026480,0.004070,0.086704,0.175439,0.450682,5.427,21408952.0,1.161864e+08


In [78]:
dfc.shape

(259, 12)

In [79]:
dfc.dtypes

code        object
name        object
company     object
source      object
5_pct      float64
20_pct     float64
60_pct     float64
120_pct    float64
250_pct    float64
close      float64
volume     float64
amount     float64
dtype: object

In [81]:
dfc[dfc.amount>=1e8].shape

(43, 12)

In [ ]:
dfc.sort_values('250_pct', ascending=False)

In [83]:
dfc[dfc.amount>=1e8].sort_values('250_pct', ascending=False)

,code,name,company,source,5_pct,20_pct,60_pct,120_pct,250_pct,close,volume,amount
216,sz159949,创业板50,华安基金管理有限公司,sz,0.024675,0.125535,0.268489,0.384211,0.707792,0.789,1.479605e+09,1.167408e+09
245,sz159915,创业板,易方达基金管理有限公司,sz,0.033199,0.112074,0.232893,0.309949,0.625000,2.054,5.724038e+08,1.175717e+09
158,sh513100,纳指ETF,国泰基金管理有限公司,sh,-0.094830,0.114076,0.248029,0.352920,0.557606,3.799,6.510399e+07,2.473301e+08
258,sz159901,深100ETF,易方达基金管理有限公司,sz,0.026480,0.004070,0.086704,0.175439,0.450682,5.427,2.140895e+07,1.161864e+08
89,sh512660,军工ETF,国泰基金管理有限公司,sh,0.082174,0.089059,0.170999,0.116037,0.291101,0.856,2.707017e+08,2.317206e+08
240,sz159922,500ETF,嘉实基金管理有限公司,sz,0.035627,0.014369,0.130385,0.159109,0.290252,5.930,3.578000e+07,2.121754e+08
76,sh512500,中证500,华夏基金管理有限公司,sh,0.036304,0.015816,0.131758,0.151120,0.289234,2.826,6.312249e+07,1.783842e+08
42,sh510500,500ETF,南方基金管理股份有限公司,sh,0.035148,0.013362,0.129585,0.149271,0.285926,6.067,2.725918e+08,1.653814e+09
157,sh513050,中概互联,易方达基金管理有限公司,sh,-0.011118,-0.011118,0.170279,0.240361,0.254772,1.512,9.483830e+07,1.433955e+08
178,sh518880,黄金ETF,华安基金管理有限公司,sh,0.020630,0.046723,0.082675,0.047031,0.254225,3.562,4.071301e+08,1.450197e+09
